In [20]:
import pandas as pd
url = '~/Documents/Expedia/train.csv'
expedia = pd.read_csv(url, parse_dates=[0, 11, 12], nrows = 100000) #import 1M rows of training data

expedia.isnull().any() # show columns with null values


date_time                    False
site_name                    False
posa_continent               False
user_location_country        False
user_location_region         False
user_location_city           False
orig_destination_distance     True
user_id                      False
is_mobile                    False
is_package                   False
channel                      False
srch_ci                       True
srch_co                       True
srch_adults_cnt              False
srch_children_cnt            False
srch_rm_cnt                  False
srch_destination_id          False
srch_destination_type_id     False
is_booking                   False
cnt                          False
hotel_continent              False
hotel_country                False
hotel_market                 False
hotel_cluster                False
dtype: bool

In [21]:
#look at the subset of rows with null in the two srch date columns
print (expedia[expedia['srch_ci'].isnull() | expedia['srch_co'].isnull()][['srch_ci', 'srch_co']])


      srch_ci srch_co
1671      NaT     NaT
2923      NaT     NaT
6040      NaT     NaT
8115      NaT     NaT
8672      NaT     NaT
8673      NaT     NaT
8674      NaT     NaT
10631     NaT     NaT
14175     NaT     NaT
15410     NaT     NaT
15411     NaT     NaT
18511     NaT     NaT
21005     NaT     NaT
21856     NaT     NaT
23996     NaT     NaT
24272     NaT     NaT
27358     NaT     NaT
29608     NaT     NaT
29693     NaT     NaT
32240     NaT     NaT
32878     NaT     NaT
33166     NaT     NaT
34197     NaT     NaT
34198     NaT     NaT
35226     NaT     NaT
35458     NaT     NaT
35577     NaT     NaT
42753     NaT     NaT
48862     NaT     NaT
51407     NaT     NaT
...       ...     ...
66488     NaT     NaT
66489     NaT     NaT
69355     NaT     NaT
70346     NaT     NaT
70498     NaT     NaT
70740     NaT     NaT
70745     NaT     NaT
74125     NaT     NaT
75872     NaT     NaT
75953     NaT     NaT
75954     NaT     NaT
76819     NaT     NaT
76900     NaT     NaT
77345     

In [22]:
# If one srch date is null, the other one is also
# Print the lengths of the sub-dataframes where columns are null to see how much overlap

print "Rows with srch columns null: " + str(len(expedia[expedia['srch_ci'].isnull() | expedia['srch_co'].isnull()]))
print "Rows with distance column null: " + str(len(expedia[expedia['orig_destination_distance'].isnull()]))
print "Rows with at either srch or distance columns null: " + str(len(expedia[expedia['orig_destination_distance'].isnull() | expedia['srch_ci'].isnull() | expedia['srch_co'].isnull()]))

Rows with srch columns null: 71
Rows with distance column null: 36922
Rows with at either srch or distance columns null: 36977


In [23]:
# drop rows where srch columns are null
expedia = expedia[expedia['srch_ci'].notnull() & expedia['srch_co'].notnull()]
len(expedia) #should be 999,001

99929

In [24]:
#Fill in null values in distance column with the mean distance, with mean grouped by posa_continent
#Other more logical "group by" values didn't work since too many unique origins with missing distance

expedia['orig_destination_distance'].fillna(expedia.groupby('posa_continent')['orig_destination_distance'].transform('mean'), inplace=True)

expedia[expedia['orig_destination_distance'].isnull()] #verify there are no more nulls


,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,...,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster


In [25]:
# Read in destinations file, and join the data with the main dataframe
# "left" join will keep all rows in Expedia, and will create nulls when the destination data is missing
destinations = pd.read_csv('~/Documents/Expedia/destinations.csv')
expedia = pd.merge(expedia, destinations, on='srch_destination_id', how='left')
expedia

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,...,d140,d141,d142,d143,d144,d145,d146,d147,d148,d149
0,2014-08-11 07:46:59,2,3,66,348,48862,2234.264100,12,0,1,...,-2.384553,-2.345528,-2.396591,-2.399953,-2.388116,-2.394294,-2.400667,-2.398716,-2.386585,-2.390370
1,2014-08-11 08:22:12,2,3,66,348,48862,2234.264100,12,0,1,...,-2.384553,-2.345528,-2.396591,-2.399953,-2.388116,-2.394294,-2.400667,-2.398716,-2.386585,-2.390370
2,2014-08-11 08:24:33,2,3,66,348,48862,2234.264100,12,0,0,...,-2.384553,-2.345528,-2.396591,-2.399953,-2.388116,-2.394294,-2.400667,-2.398716,-2.386585,-2.390370
3,2014-08-09 18:05:16,2,3,66,442,35390,913.193200,93,0,0,...,-2.254121,-2.242363,-2.299968,-2.299212,-2.286866,-2.261816,-2.299968,-2.295632,-2.299968,-2.299968
4,2014-08-09 18:08:18,2,3,66,442,35390,913.625900,93,0,0,...,-2.254121,-2.242363,-2.299968,-2.299212,-2.286866,-2.261816,-2.299968,-2.295632,-2.299968,-2.299968
5,2014-08-09 18:13:12,2,3,66,442,35390,911.514200,93,0,0,...,-2.254121,-2.242363,-2.299968,-2.299212,-2.286866,-2.261816,-2.299968,-2.295632,-2.299968,-2.299968
6,2014-07-16 09:42:23,2,3,66,189,10067,1850.663392,501,0,0,...,-2.355353,-2.310370,-2.356791,-2.356312,-2.337682,-2.355353,-2.356368,-2.353851,-2.356791,-2.294829
7,2014-07-16 09:45:48,2,3,66,189,10067,1850.663392,501,0,1,...,-2.355353,-2.310370,-2.356791,-2.356312,-2.337682,-2.355353,-2.356368,-2.353851,-2.356791,-2.294829
8,2014-07-16 09:52:11,2,3,66,189,10067,1850.663392,501,0,0,...,-2.355353,-2.310370,-2.356791,-2.356312,-2.337682,-2.355353,-2.356368,-2.353851,-2.356791,-2.294829
9,2014-07-16 09:55:24,2,3,66,189,10067,1850.663392,501,0,0,...,-2.355353,-2.310370,-2.356791,-2.356312,-2.337682,-2.355353,-2.356368,-2.353851,-2.356791,-2.294829


In [26]:
expedia.isnull().any()

date_time                    False
site_name                    False
posa_continent               False
user_location_country        False
user_location_region         False
user_location_city           False
orig_destination_distance    False
user_id                      False
is_mobile                    False
is_package                   False
channel                      False
srch_ci                      False
srch_co                      False
srch_adults_cnt              False
srch_children_cnt            False
srch_rm_cnt                  False
srch_destination_id          False
srch_destination_type_id     False
is_booking                   False
cnt                          False
hotel_continent              False
hotel_country                False
hotel_market                 False
hotel_cluster                False
d1                            True
d2                            True
d3                            True
d4                            True
d5                  

In [27]:
for colnum in range(1, 150): # fill null d-values with column means, grouped by hotel_continent
    colname = 'd' + str(colnum)
    expedia[[colname]].fillna(expedia.groupby('hotel_continent')[[colname]].transform('mean'), inplace=True)
 
expedia.isnull().any()

date_time                    False
site_name                    False
posa_continent               False
user_location_country        False
user_location_region         False
user_location_city           False
orig_destination_distance    False
user_id                      False
is_mobile                    False
is_package                   False
channel                      False
srch_ci                      False
srch_co                      False
srch_adults_cnt              False
srch_children_cnt            False
srch_rm_cnt                  False
srch_destination_id          False
srch_destination_type_id     False
is_booking                   False
cnt                          False
hotel_continent              False
hotel_country                False
hotel_market                 False
hotel_cluster                False
d1                            True
d2                            True
d3                            True
d4                            True
d5                  

In [28]:
len(expedia[expedia['d1'].isnull()]) #how many more rows with nulls, if d1 is null, all d values are null

608

In [29]:
expedia = expedia[expedia['d1'].notnull()] # drop remaining rows with null d values
expedia.isnull().any()


date_time                    False
site_name                    False
posa_continent               False
user_location_country        False
user_location_region         False
user_location_city           False
orig_destination_distance    False
user_id                      False
is_mobile                    False
is_package                   False
channel                      False
srch_ci                      False
srch_co                      False
srch_adults_cnt              False
srch_children_cnt            False
srch_rm_cnt                  False
srch_destination_id          False
srch_destination_type_id     False
is_booking                   False
cnt                          False
hotel_continent              False
hotel_country                False
hotel_market                 False
hotel_cluster                False
d1                           False
d2                           False
d3                           False
d4                           False
d5                  

In [30]:
#nulls are gone, now let's make some features
from datetime import datetime
import numpy as np

expedia['duration'] = (expedia['srch_co'] - expedia['srch_ci']) / np.timedelta64(1, 'D') # duration of trip
expedia['in_advance'] = (expedia['srch_ci'] - expedia['date_time']) / np.timedelta64(1, 'D') # time booked in advance
expedia['trip_month'] = pd.DatetimeIndex(expedia['srch_ci']).month # month of the check-in date

expedia

C:\Users\Brian\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Brian\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Brian\Anaconda2\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,...,d143,d144,d145,d146,d147,d148,d149,duration,in_advance,trip_month
0,2014-08-11 07:46:59,2,3,66,348,48862,2234.264100,12,0,1,...,-2.399953,-2.388116,-2.394294,-2.400667,-2.398716,-2.386585,-2.390370,4.0,15.675706,8
1,2014-08-11 08:22:12,2,3,66,348,48862,2234.264100,12,0,1,...,-2.399953,-2.388116,-2.394294,-2.400667,-2.398716,-2.386585,-2.390370,4.0,17.651250,8
2,2014-08-11 08:24:33,2,3,66,348,48862,2234.264100,12,0,0,...,-2.399953,-2.388116,-2.394294,-2.400667,-2.398716,-2.386585,-2.390370,4.0,17.649618,8
3,2014-08-09 18:05:16,2,3,66,442,35390,913.193200,93,0,0,...,-2.299212,-2.286866,-2.261816,-2.299968,-2.295632,-2.299968,-2.299968,5.0,105.246343,11
4,2014-08-09 18:08:18,2,3,66,442,35390,913.625900,93,0,0,...,-2.299212,-2.286866,-2.261816,-2.299968,-2.295632,-2.299968,-2.299968,5.0,105.244236,11
5,2014-08-09 18:13:12,2,3,66,442,35390,911.514200,93,0,0,...,-2.299212,-2.286866,-2.261816,-2.299968,-2.295632,-2.299968,-2.299968,5.0,105.240833,11
6,2014-07-16 09:42:23,2,3,66,189,10067,1850.663392,501,0,0,...,-2.356312,-2.337682,-2.355353,-2.356368,-2.353851,-2.356791,-2.294829,1.0,15.595567,8
7,2014-07-16 09:45:48,2,3,66,189,10067,1850.663392,501,0,1,...,-2.356312,-2.337682,-2.355353,-2.356368,-2.353851,-2.356791,-2.294829,1.0,15.593194,8
8,2014-07-16 09:52:11,2,3,66,189,10067,1850.663392,501,0,0,...,-2.356312,-2.337682,-2.355353,-2.356368,-2.353851,-2.356791,-2.294829,1.0,15.588762,8
9,2014-07-16 09:55:24,2,3,66,189,10067,1850.663392,501,0,0,...,-2.356312,-2.337682,-2.355353,-2.356368,-2.353851,-2.356791,-2.294829,1.0,15.586528,8


In [31]:
expedia.describe()


,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,...,d143,d144,d145,d146,d147,d148,d149,duration,in_advance,trip_month
count,99321.000000,99321.000000,99321.000000,99321.000000,99321.000000,99321.000000,99321.000000,99321.000000,99321.000000,99321.000000,...,99321.000000,99321.000000,99321.000000,99321.000000,99321.000000,99321.000000,99321.000000,99321.000000,99321.000000,99321.000000
mean,9.076268,2.638234,84.449412,311.322600,28440.290301,2003.037554,195446.352212,0.138098,0.260106,5.763897,...,-2.254346,-2.248540,-2.257033,-2.260938,-2.267019,-2.265801,-2.239062,3.385105,54.123924,7.200109
std,12.055419,0.751119,54.263845,209.287157,16825.503789,1707.969987,110086.223927,0.345004,0.438694,3.770550,...,0.062957,0.060402,0.049961,0.068761,0.050916,0.052475,0.070997,3.128750,64.602090,3.372205
min,2.000000,0.000000,0.000000,0.000000,3.000000,0.005600,12.000000,0.000000,0.000000,0.000000,...,-2.399953,-2.403371,-2.394729,-2.403371,-2.398716,-2.388346,-2.392952,-234.000000,-107.795289,1.000000
25%,2.000000,3.000000,66.000000,174.000000,13894.000000,715.748200,107438.000000,0.000000,0.000000,2.000000,...,-2.286122,-2.278273,-2.283220,-2.293691,-2.293223,-2.294888,-2.276540,1.000000,9.162940,4.000000
50%,2.000000,3.000000,66.000000,311.000000,27733.000000,1850.663392,181651.000000,0.000000,0.000000,9.000000,...,-2.249634,-2.242573,-2.250099,-2.258193,-2.261589,-2.260641,-2.240363,3.000000,29.720174,8.000000
75%,11.000000,3.000000,69.000000,385.000000,43040.000000,2504.137670,300773.000000,0.000000,1.000000,9.000000,...,-2.219957,-2.211198,-2.224354,-2.225821,-2.232612,-2.230841,-2.204944,4.000000,75.506979,10.000000
max,53.000000,4.000000,239.000000,1025.000000,56495.000000,11641.224200,391007.000000,1.000000,1.000000,10.000000,...,-1.930367,-1.935139,-2.012649,-1.463392,-1.939302,-1.945084,-1.814839,190.000000,557.609433,12.000000


In [32]:
df_channel = pd.get_dummies(expedia['channel'], prefix='channel')
df_tripmonth = pd.get_dummies(expedia['trip_month'], prefix='month')
df_sitename = pd.get_dummies(expedia['site_name'], prefix='site')
df_posacontinent = pd.get_dummies(expedia['posa_continent'], prefix='posa')


In [38]:
# create feature matrix X

# excluded columns 'site_name', 'posa_continent', 'user_location_country', 'user_location_region', 'hotel_continent', 'hotel_country', 
# 'user_location_city',  'user_id', 'hotel_market', 'srch_destination_type_id', 'is_booking', 'cnt'

feature_cols = [ 
                'is_mobile', 'is_package', 'srch_adults_cnt', 'srch_children_cnt', 
                'orig_destination_distance', 'srch_rm_cnt',   
                'duration', 'in_advance']

X = expedia[expedia['is_booking' == 1]]
X = X[feature_cols]

X = X.join(df_channel)
X = X.join(df_tripmonth)
# X = X.join(df_sitename)
# X = X.join(df_posacontinent)

# X = X.join(expedia.iloc[:,24:173]) # add in the d-value columns

print(X.shape)

# create response vector (y)
y = expedia[expedia['is_booking' == 1]].hotel_cluster

print(y.shape)

KeyError: False

In [34]:
print "Null accuracy: " + str(y.value_counts(1).head(1))


Null accuracy: 91    0.027869
Name: hotel_cluster, dtype: float64


In [35]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

print X_train.shape
print X_test.shape

(74490, 76)
(24831, 76)


In [36]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# standardize X_train
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [37]:
# import the classifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

# instantiate the model
knn = KNeighborsClassifier(n_neighbors=10) # try with 10 first

# train the model
knn.fit(X_train_scaled, y_train)

y_pred_class = knn.predict(X_test_scaled)

print metrics.accuracy_score(y_test, y_pred_class) # This is the accuracy

0.110426483025


In [22]:
pd.pivot_table(expedia, index=['srch_destination_id'], columns=['hotel_cluster'], values='cnt', aggfunc='count', fill_value=0)
#hotels do not stay in the same hotel_cluster....they move around

hotel_cluster,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
srch_destination_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,13,0,0,...,0,2,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,5,0,0,0,0,0
14,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16,0,0,0,0,0,0,0,9,0,0,...,0,0,0,0,0,0,0,0,0,0
18,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19,0,0,0,1,8,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
